In [ ]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain-core langchain_openai python-dotenv langsmith pydantic spotipy

In [ ]:
%pip install --quiet -U jupyterlab-lsp
%pip install --quiet -U "python-lsp-server[all]"

In [139]:
## Setup logging
import logging
import os
from dotenv import load_dotenv

load_dotenv(override=True)
logger = logging.getLogger(__name__)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',  # Define the format
    handlers=[logging.StreamHandler()]  # Output to the console
)

# Define Spotify URI type

In [140]:
from typing import List, NewType

# Define a custom URI type
SpotifyURI = NewType('SpotifyURI', str)

# Define Spotify Track and Playlist Models

In [141]:
from typing import List, Dict, Optional
from pydantic import BaseModel, Field

# Define a Pydantic model for Playlist
class Playlist(BaseModel):
    """
    A model representing a Spotify playlist.
    """
    id: str = Field(..., description="The unique identifier for the playlist.")
    uri: str = Field(..., description="The Spotify URI for the playlist.")
    name: str = Field(..., description="The name of the playlist.")
    description: Optional[str] = Field(None, description="The playlist's description.")
    owner: Optional[str] = Field(None, description="The display name of the playlist's owner.")
    tracks_total: Optional[int] = Field(None, description="The total number of tracks in the playlist.")
    is_public: Optional[bool] = Field(None, description="Indicates if the playlist is public.")
    collaborative: Optional[bool] = Field(None, description="Indicates if the playlist is collaborative.")
    snapshot_id: Optional[str] = Field(None, description="The version identifier for the current playlist.")

# Define a Pydantic model for Track
class Track(BaseModel):
    """
    A model representing a Spotify track.
    """
    id: str = Field(..., description="The unique identifier for the track.")
    uri: str = Field(..., description="The Spotify URI for the track.")
    name: str = Field(..., description="The name of the track.")
    artists: List[str] = Field(..., description="A list of artists who performed the track.")
    album: str = Field(..., description="The name of the album the track is from.")
    duration_ms: Optional[int] = Field(None, description="The track length in milliseconds.")
    explicit: Optional[bool] = Field(None, description="Indicates if the track has explicit content.")
    popularity: Optional[int] = Field(None, description="The popularity of the track (0-100).")


# Define a Pydantic model for Tracks
class Tracks(BaseModel):
    """
    A model representing a collection of Spotify tracks.
    """
    tracks: List[Track] = Field(..., description="A list of Track objects.")
    total: Optional[int] = Field(None, description="Total number of tracks in the collection.")
    playlist_name: Optional[str] = Field(None, description="Name of the playlist.")

    class Config:
        """
        Configuration for the Tracks model.
        """
        str_strip_whitespace = True

In [142]:
from typing import Dict, List, Any, Annotated
from typing import Annotated
from typing_extensions import TypedDict
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage, ToolMessage
from langgraph.graph.message import add_messages


class State(TypedDict):
    """
    Represents the state of the conversation and Spotify information

    Attributes:
        playlists (List[Playlist]): A list of Spotify Playlists.
        tracks (List[Track]): Track list for a Spotify Playlist
        new_playlist: (Playlist) : New Spotify playlist data
        new_tracks: (List[Track]): Tracks for the new playlist
    """
    new_playlist: Playlist
    new_tracks: List[Track]
    playlists: List[Playlist]
    tracks: List[Track]
    messages: Annotated[List[BaseMessage], add_messages]


# Define Spotify Tools

In [143]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from langchain_core.tools import tool
from typing import List, NewType
import json


def get_spotify_client() -> spotipy.Spotify:
    """
    Initializes and returns a Spotify client with user authentication.

    Returns:
        spotipy.Spotify: An authenticated Spotify client.
    """
    auth_manager = SpotifyClientCredentials(
        client_id=os.environ.get("SPOTIFY_CLIENT_ID"),
        client_secret=os.environ.get("SPOTIFY_CLIENT_SECRET"),
    )
    return spotipy.Spotify(auth_manager=auth_manager)


@tool
def get_playlists() -> Dict[str, List[Playlist]]:
    """
    Retrieves a list of the user's Spotify playlists.

    Returns:
        Dict[str, List[Playlist]]: A dictionary containing a list of Playlist models under the 'playlists' key.
    """
    sp = get_spotify_client()
    playlists: List[Playlist] = []

    try:
        # Fetch the current user's playlists with pagination
        playlists_raw = sp.user_playlists(user=os.getenv("SPOTIFY_USER_ID"), limit=100)
        while playlists_raw:
            for playlist_data in playlists_raw['items']:
                # Map API data to the Playlist model
                playlist = Playlist(
                    id=playlist_data['id'],
                    uri=playlist_data['uri'],
                    name=playlist_data['name'],
                    description=playlist_data.get('description'),
                    owner=playlist_data['owner']['display_name'],
                    tracks_total=playlist_data['tracks']['total'],
                    is_public=playlist_data.get('public'),
                    collaborative=playlist_data.get('collaborative'),
                    snapshot_id=playlist_data.get('snapshot_id')
                )
                playlists.append(playlist)
            # Check if there is a next page
            if playlists_raw['next']:
                playlists_raw = sp.next(playlists_raw)
            else:
                break
    except spotipy.SpotifyException as e:
        return {"error": str(e)}

    # Serialize the playlists to JSON-serializable dictionaries
    serialized_playlists = [playlist.model_dump() for playlist in playlists]
    return {"playlists": serialized_playlists}


@tool
def get_track_list(playlist_id: str) -> Dict[str, List[Track]]:
    """
    Retrieves the track list for a specific Spotify playlist.

    Args:
        playlist_id (str): The unique identifier of the Spotify playlist.

    Returns:
        Dict[str, List[Track]]: A dictionary containing a list of Track models under the 'tracks' key.
    """
    sp = get_spotify_client()
    playlist_tracks: List[Track] = []

    try:
        # Fetch the playlist's tracks with pagination
        playlist = sp.user_playlist(user=os.getenv("SPOTIFY_USER_ID"), playlist_id=playlist_id)
        if 'tracks' in playlist:
            tracks = playlist["tracks"]
            while tracks:
                for item in tracks['items']:
                    track_data = item['track']
                    # Map API data to the Track model
                    track = Track(
                        id=track_data['id'],
                        uri=track_data['uri'],
                        name=track_data['name'],
                        artists=[artist['name'] for artist in track_data['artists']],
                        album=track_data['album']['name'],
                        duration_ms=track_data.get('duration_ms'),
                        explicit=track_data.get('explicit'),
                        popularity=track_data.get('popularity')
                    )
                    playlist_tracks.append(track)
                # Check if there is a next page
                if tracks['next']:
                    # TODO unclear in this case
                    tracks = sp.next(tracks)
                else:
                    break
    except spotipy.SpotifyException as e:
        return {"error": str(e)}

    # Serialize the tracks to JSON-serializable dictionaries
    serialized_tracks = [track.model_dump() for track in playlist_tracks]
    return {"tracks": serialized_tracks}


@tool
def create_spotify_playlist(name: str, description: str = "Agentic Playlist") -> Dict[str, Playlist]:
    """
    Creates a new playlist on Spotify.

    This function only creates the playlist; it does not add tracks.

    Args:
        name (str): The name of the new playlist.
        description (str, optional): The description of the playlist.

    Returns:
        Dict[str, Playlist]: A dictionary containing the new Playlist model under the 'new_playlist' key.
    """
    sp = get_spotify_client()
    try:
        # Retrieve the current user's ID
        user_id = sp.current_user()['id']
        # Create a new playlist
        new_playlist_data = sp.user_playlist_create(
            user=user_id,
            name=name,
            public=True,
            description=description
        )
        # Map API data to the Playlist model
        new_playlist = Playlist(
            id=new_playlist_data['id'],
            uri=new_playlist_data['uri'],
            name=new_playlist_data['name'],
            description=new_playlist_data.get('description'),
            owner=new_playlist_data['owner']['display_name'],
            tracks_total=new_playlist_data['tracks']['total'],
            is_public=new_playlist_data.get('public'),
            collaborative=new_playlist_data.get('collaborative'),
            snapshot_id=new_playlist_data.get('snapshot_id')
        )
    except spotipy.SpotifyException as e:
        return {"error": str(e)}
    return {"new_playlist": new_playlist.model_dump()}

@tool
def add_tracks_to_playlist(playlist_id: str, tracks: List[SpotifyURI]) -> Dict[str, Any]:
    """
    Adds tracks to a Spotify playlist.

    Args:
        playlist_id (str): ID of the playlist.
        tracks (List[SpotifyURI]): List of Spotify track URIs.

    Returns:
        Dict[str, Any]: A dictionary indicating success or error.
    """
    sp = get_spotify_client()
    try:
        sp.playlist_add_items(playlist_id=playlist_id, items=tracks)
    except spotipy.SpotifyException as e:
        return {"error": str(e)}
    return {"success": True} 



# Define Search Tool

In [144]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(
    max_results=50,
    include_answer=True,
    include_raw_content=True,
    include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
)
name = search_tool.get_name()
desc = search_tool.description
desc

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

# Create ToolNode

In [145]:
from langgraph.prebuilt import ToolNode

tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist]
# tools = [get_playlists, get_track_list, create_spotify_playlist, add_tracks_to_playlist, search_tool]
tool_node = ToolNode(tools)

# Manual Test of Tool Infra

In [146]:
from langchain_core.messages import AIMessage, HumanMessage
message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "get_playlists",
            "args" : {},
            "id": "tool_call_id",
            "type": "tool_call",
        }
    ],
)

# tool_node.invoke({"messages": [message_with_single_tool_call]})

# Bind Tools to Model

In [168]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=os.getenv("OPENAI_MODEL_NAME"))
llm_with_tools = llm.bind_tools(tools)

# Define Prompt

In [189]:
prompt = """
Create and execute a step-by-step plan with atomic tasks to solve the following problem:

Build a Spotify Playlist with tracks of the same vibe and genres as my 'New Rock and Blues' playlist. Here are the rules you must follow.

1 - Only suggest tracks from artist not present on the 'New Rock and Blues' playlist
2 - Artist should have achieved success after year 2015
3 - Do not use 'The Raconteurs' as an artist in the new playlist
4 - Rely on your training data as a music recommendation system.
5 - Rely on your training data and best practices to arrange the tracks to create a smooth listening experience, considering tempo, energy, and mood.
6 - New playlist should have no less than 100 tracks

These steps can be repeated any number of times
"""

# First Message

In [190]:
from langchain_core.messages import AIMessage, HumanMessage

human_message = HumanMessage(prompt)
ai_tool_call_message = llm_with_tools.invoke([human_message])
ai_tool_call_message


2024-11-12 00:16:22,024 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='To build a Spotify playlist that meets the given criteria, we\'ll execute a series of atomic tasks. Here\'s the step-by-step plan:\n\n### Step 1: Identify Existing Tracks and Artists\n\n1. **Retrieve \'New Rock and Blues\' Playlist Details**\n   - Get the list of tracks from the \'New Rock and Blues\' playlist.\n   - Extract the list of artists present in this playlist to ensure we don\'t include them in the new playlist.\n\n### Step 2: Collect New Tracks\n\n2. **Research and Suggest New Tracks**\n   - Use training data knowledge to identify artists who achieved success after 2015 in the Rock and Blues genres.\n   - Exclude \'The Raconteurs\' and artists already present in the \'New Rock and Blues\' playlist.\n\n3. **Select Tracks for Smooth Listening Experience**\n   - Arrange selected tracks considering tempo, energy, and mood to ensure a cohesive listening experience.\n\n### Step 3: Create the New Playlist\n\n4. **Create a New Playlist on Spotify**\n   - Name: "Ne

In [191]:
print(ai_tool_call_message.content)

To build a Spotify playlist that meets the given criteria, we'll execute a series of atomic tasks. Here's the step-by-step plan:

### Step 1: Identify Existing Tracks and Artists

1. **Retrieve 'New Rock and Blues' Playlist Details**
   - Get the list of tracks from the 'New Rock and Blues' playlist.
   - Extract the list of artists present in this playlist to ensure we don't include them in the new playlist.

### Step 2: Collect New Tracks

2. **Research and Suggest New Tracks**
   - Use training data knowledge to identify artists who achieved success after 2015 in the Rock and Blues genres.
   - Exclude 'The Raconteurs' and artists already present in the 'New Rock and Blues' playlist.

3. **Select Tracks for Smooth Listening Experience**
   - Arrange selected tracks considering tempo, energy, and mood to ensure a cohesive listening experience.

### Step 3: Create the New Playlist

4. **Create a New Playlist on Spotify**
   - Name: "New Rock and Blues Vibes"
   - Description: A curate

# chat_prompt_template holds all the messages (Human, AI, Tool)

In [192]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt_template: ChatPromptTemplate = human_message + ai_tool_call_message
chat_prompt_template.format_messages()

[HumanMessage(content="\nCreate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\nBuild a Spotify Playlist with tracks of the same vibe and genres as my 'New Rock and Blues' playlist. Here are the rules you must follow.\n\n1 - Only suggest tracks from artist not present on the 'New Rock and Blues' playlist\n2 - Artist should have achieved success after year 2015\n3 - Do not use 'The Raconteurs' as an artist in the new playlist\n4 - Rely on your training data as a music recommendation system.\n5 - Rely on your training data and best practices to arrange the tracks to create a smooth listening experience, considering tempo, energy, and mood.\n6 - New playlist should have no less than 100 tracks\n\nThese steps can be repeated any number of times\n", additional_kwargs={}, response_metadata={}),
 AIMessage(content='To build a Spotify playlist that meets the given criteria, we\'ll execute a series of atomic tasks. Here\'s the step-by-step plan:\n\n### Step 

# Tool Call (get_playlists)

In [193]:
import json
from langchain_core.messages import ToolMessage

response = tool_node.invoke({"messages": [ai_tool_call_message]})
response


{'messages': [ToolMessage(content='{"playlists": [{"id": "2dbYK5b7J0F7IdH5n1TEUK", "uri": "spotify:playlist:2dbYK5b7J0F7IdH5n1TEUK", "name": "RPreacher", "description": "", "owner": "Ray", "tracks_total": 3, "is_public": true, "collaborative": false, "snapshot_id": "AAAABG1lhIiRJT/NE9RpP4K6UnmZWixo"}, {"id": "4ack9YtUhdxRayJDAqlfQe", "uri": "spotify:playlist:4ack9YtUhdxRayJDAqlfQe", "name": "RP Bossa Nova Chill ", "description": "", "owner": "Ray", "tracks_total": 30, "is_public": true, "collaborative": false, "snapshot_id": "AAAAIThdM9g/YPQwj7nuoHhewWyLoKEi"}, {"id": "75NW18NgdZuZeifrcjxKlZ", "uri": "spotify:playlist:75NW18NgdZuZeifrcjxKlZ", "name": "GVF", "description": "", "owner": "Ray", "tracks_total": 11, "is_public": true, "collaborative": false, "snapshot_id": "AAAADKZNV9h+cfi1PbOi6yjNHMe8mLWC"}, {"id": "5iyONtUO21O88xw8pBblwh", "uri": "spotify:playlist:5iyONtUO21O88xw8pBblwh", "name": "Now And Then", "description": "", "owner": "Ray", "tracks_total": 7, "is_public": true, "col

In [194]:
message_list: List[ToolMessage] = response["messages"]
playlist_tool_message: ToolMessage = message_list[0]
playlist_tool_message

ToolMessage(content='{"playlists": [{"id": "2dbYK5b7J0F7IdH5n1TEUK", "uri": "spotify:playlist:2dbYK5b7J0F7IdH5n1TEUK", "name": "RPreacher", "description": "", "owner": "Ray", "tracks_total": 3, "is_public": true, "collaborative": false, "snapshot_id": "AAAABG1lhIiRJT/NE9RpP4K6UnmZWixo"}, {"id": "4ack9YtUhdxRayJDAqlfQe", "uri": "spotify:playlist:4ack9YtUhdxRayJDAqlfQe", "name": "RP Bossa Nova Chill ", "description": "", "owner": "Ray", "tracks_total": 30, "is_public": true, "collaborative": false, "snapshot_id": "AAAAIThdM9g/YPQwj7nuoHhewWyLoKEi"}, {"id": "75NW18NgdZuZeifrcjxKlZ", "uri": "spotify:playlist:75NW18NgdZuZeifrcjxKlZ", "name": "GVF", "description": "", "owner": "Ray", "tracks_total": 11, "is_public": true, "collaborative": false, "snapshot_id": "AAAADKZNV9h+cfi1PbOi6yjNHMe8mLWC"}, {"id": "5iyONtUO21O88xw8pBblwh", "uri": "spotify:playlist:5iyONtUO21O88xw8pBblwh", "name": "Now And Then", "description": "", "owner": "Ray", "tracks_total": 7, "is_public": true, "collaborative": f

In [195]:
json.loads(playlist_tool_message.content)

{'playlists': [{'id': '2dbYK5b7J0F7IdH5n1TEUK',
   'uri': 'spotify:playlist:2dbYK5b7J0F7IdH5n1TEUK',
   'name': 'RPreacher',
   'description': '',
   'owner': 'Ray',
   'tracks_total': 3,
   'is_public': True,
   'collaborative': False,
   'snapshot_id': 'AAAABG1lhIiRJT/NE9RpP4K6UnmZWixo'},
  {'id': '4ack9YtUhdxRayJDAqlfQe',
   'uri': 'spotify:playlist:4ack9YtUhdxRayJDAqlfQe',
   'name': 'RP Bossa Nova Chill ',
   'description': '',
   'owner': 'Ray',
   'tracks_total': 30,
   'is_public': True,
   'collaborative': False,
   'snapshot_id': 'AAAAIThdM9g/YPQwj7nuoHhewWyLoKEi'},
  {'id': '75NW18NgdZuZeifrcjxKlZ',
   'uri': 'spotify:playlist:75NW18NgdZuZeifrcjxKlZ',
   'name': 'GVF',
   'description': '',
   'owner': 'Ray',
   'tracks_total': 11,
   'is_public': True,
   'collaborative': False,
   'snapshot_id': 'AAAADKZNV9h+cfi1PbOi6yjNHMe8mLWC'},
  {'id': '5iyONtUO21O88xw8pBblwh',
   'uri': 'spotify:playlist:5iyONtUO21O88xw8pBblwh',
   'name': 'Now And Then',
   'description': '',
   'ow

In [196]:
# We recreate the ToolMessage to take care of UNicode characters
tool_message = ToolMessage(content=playlist_tool_message.content, name=playlist_tool_message.name, tool_call_id=playlist_tool_message.tool_call_id)
tool_message

ToolMessage(content='{"playlists": [{"id": "2dbYK5b7J0F7IdH5n1TEUK", "uri": "spotify:playlist:2dbYK5b7J0F7IdH5n1TEUK", "name": "RPreacher", "description": "", "owner": "Ray", "tracks_total": 3, "is_public": true, "collaborative": false, "snapshot_id": "AAAABG1lhIiRJT/NE9RpP4K6UnmZWixo"}, {"id": "4ack9YtUhdxRayJDAqlfQe", "uri": "spotify:playlist:4ack9YtUhdxRayJDAqlfQe", "name": "RP Bossa Nova Chill ", "description": "", "owner": "Ray", "tracks_total": 30, "is_public": true, "collaborative": false, "snapshot_id": "AAAAIThdM9g/YPQwj7nuoHhewWyLoKEi"}, {"id": "75NW18NgdZuZeifrcjxKlZ", "uri": "spotify:playlist:75NW18NgdZuZeifrcjxKlZ", "name": "GVF", "description": "", "owner": "Ray", "tracks_total": 11, "is_public": true, "collaborative": false, "snapshot_id": "AAAADKZNV9h+cfi1PbOi6yjNHMe8mLWC"}, {"id": "5iyONtUO21O88xw8pBblwh", "uri": "spotify:playlist:5iyONtUO21O88xw8pBblwh", "name": "Now And Then", "description": "", "owner": "Ray", "tracks_total": 7, "is_public": true, "collaborative": f

# Add To message template list

In [197]:
chat_prompt_template += tool_message

# Format Message List for LLM

In [198]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt_template.format_messages()

[HumanMessage(content="\nCreate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\nBuild a Spotify Playlist with tracks of the same vibe and genres as my 'New Rock and Blues' playlist. Here are the rules you must follow.\n\n1 - Only suggest tracks from artist not present on the 'New Rock and Blues' playlist\n2 - Artist should have achieved success after year 2015\n3 - Do not use 'The Raconteurs' as an artist in the new playlist\n4 - Rely on your training data as a music recommendation system.\n5 - Rely on your training data and best practices to arrange the tracks to create a smooth listening experience, considering tempo, energy, and mood.\n6 - New playlist should have no less than 100 tracks\n\nThese steps can be repeated any number of times\n", additional_kwargs={}, response_metadata={}),
 AIMessage(content='To build a Spotify playlist that meets the given criteria, we\'ll execute a series of atomic tasks. Here\'s the step-by-step plan:\n\n### Step 

# Send Playlists Results to LLM

In [199]:
ai_tool_call_message_2 = llm_with_tools.invoke(chat_prompt_template.format_messages())

2024-11-12 00:16:23,476 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [200]:
chat_prompt_template += ai_tool_call_message_2
chat_prompt_template.format_messages()

[HumanMessage(content="\nCreate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\nBuild a Spotify Playlist with tracks of the same vibe and genres as my 'New Rock and Blues' playlist. Here are the rules you must follow.\n\n1 - Only suggest tracks from artist not present on the 'New Rock and Blues' playlist\n2 - Artist should have achieved success after year 2015\n3 - Do not use 'The Raconteurs' as an artist in the new playlist\n4 - Rely on your training data as a music recommendation system.\n5 - Rely on your training data and best practices to arrange the tracks to create a smooth listening experience, considering tempo, energy, and mood.\n6 - New playlist should have no less than 100 tracks\n\nThese steps can be repeated any number of times\n", additional_kwargs={}, response_metadata={}),
 AIMessage(content='To build a Spotify playlist that meets the given criteria, we\'ll execute a series of atomic tasks. Here\'s the step-by-step plan:\n\n### Step 

# Tool Call (get_track_list)

In [201]:
response = tool_node.invoke({"messages": [ai_tool_call_message_2]})
response

{'messages': [ToolMessage(content='{"tracks": [{"id": "59oXRNRnmmTbwgkZkV7l4g", "uri": "spotify:track:59oXRNRnmmTbwgkZkV7l4g", "name": "Low Down Rolling Stone", "artists": ["Gary Clark Jr."], "album": "This Land", "duration_ms": 258200, "explicit": false, "popularity": 46}, {"id": "2TmVlVk2t6d9MoqLNasfgS", "uri": "spotify:track:2TmVlVk2t6d9MoqLNasfgS", "name": "Take Me Away", "artists": ["Ayron Jones"], "album": "Child Of The State", "duration_ms": 246932, "explicit": true, "popularity": 41}, {"id": "5UMD1Iz7yyVK8Q5FLsVt3q", "uri": "spotify:track:5UMD1Iz7yyVK8Q5FLsVt3q", "name": "Walk On Water", "artists": ["Thirty Seconds To Mars"], "album": "Walk On Water", "duration_ms": 188227, "explicit": false, "popularity": 0}, {"id": "0Tr5G2mE56eLUGvCaXRM8I", "uri": "spotify:track:0Tr5G2mE56eLUGvCaXRM8I", "name": "No Good", "artists": ["KALEO"], "album": "A/B", "duration_ms": 234240, "explicit": false, "popularity": 64}, {"id": "7e8utCy2JlSB8dRHKi49xM", "uri": "spotify:track:7e8utCy2JlSB8dRHKi4

# Decode AI Tool Call message

In [202]:
track_list_tool_message: ToolMessage = response["messages"][0]
track_list_tool_message

ToolMessage(content='{"tracks": [{"id": "59oXRNRnmmTbwgkZkV7l4g", "uri": "spotify:track:59oXRNRnmmTbwgkZkV7l4g", "name": "Low Down Rolling Stone", "artists": ["Gary Clark Jr."], "album": "This Land", "duration_ms": 258200, "explicit": false, "popularity": 46}, {"id": "2TmVlVk2t6d9MoqLNasfgS", "uri": "spotify:track:2TmVlVk2t6d9MoqLNasfgS", "name": "Take Me Away", "artists": ["Ayron Jones"], "album": "Child Of The State", "duration_ms": 246932, "explicit": true, "popularity": 41}, {"id": "5UMD1Iz7yyVK8Q5FLsVt3q", "uri": "spotify:track:5UMD1Iz7yyVK8Q5FLsVt3q", "name": "Walk On Water", "artists": ["Thirty Seconds To Mars"], "album": "Walk On Water", "duration_ms": 188227, "explicit": false, "popularity": 0}, {"id": "0Tr5G2mE56eLUGvCaXRM8I", "uri": "spotify:track:0Tr5G2mE56eLUGvCaXRM8I", "name": "No Good", "artists": ["KALEO"], "album": "A/B", "duration_ms": 234240, "explicit": false, "popularity": 64}, {"id": "7e8utCy2JlSB8dRHKi49xM", "uri": "spotify:track:7e8utCy2JlSB8dRHKi49xM", "name": 

In [203]:
json.loads(track_list_tool_message.content)

{'tracks': [{'id': '59oXRNRnmmTbwgkZkV7l4g',
   'uri': 'spotify:track:59oXRNRnmmTbwgkZkV7l4g',
   'name': 'Low Down Rolling Stone',
   'artists': ['Gary Clark Jr.'],
   'album': 'This Land',
   'duration_ms': 258200,
   'explicit': False,
   'popularity': 46},
  {'id': '2TmVlVk2t6d9MoqLNasfgS',
   'uri': 'spotify:track:2TmVlVk2t6d9MoqLNasfgS',
   'name': 'Take Me Away',
   'artists': ['Ayron Jones'],
   'album': 'Child Of The State',
   'duration_ms': 246932,
   'explicit': True,
   'popularity': 41},
  {'id': '5UMD1Iz7yyVK8Q5FLsVt3q',
   'uri': 'spotify:track:5UMD1Iz7yyVK8Q5FLsVt3q',
   'name': 'Walk On Water',
   'artists': ['Thirty Seconds To Mars'],
   'album': 'Walk On Water',
   'duration_ms': 188227,
   'explicit': False,
   'popularity': 0},
  {'id': '0Tr5G2mE56eLUGvCaXRM8I',
   'uri': 'spotify:track:0Tr5G2mE56eLUGvCaXRM8I',
   'name': 'No Good',
   'artists': ['KALEO'],
   'album': 'A/B',
   'duration_ms': 234240,
   'explicit': False,
   'popularity': 64},
  {'id': '7e8utCy2J

# Create Track List ToolMessage for LLM

In [204]:
# We recreate the ToolMessage to take care of UNicode characters
tool_message = ToolMessage(content=track_list_tool_message.content, name=track_list_tool_message.name, tool_call_id=track_list_tool_message.tool_call_id)
tool_message

ToolMessage(content='{"tracks": [{"id": "59oXRNRnmmTbwgkZkV7l4g", "uri": "spotify:track:59oXRNRnmmTbwgkZkV7l4g", "name": "Low Down Rolling Stone", "artists": ["Gary Clark Jr."], "album": "This Land", "duration_ms": 258200, "explicit": false, "popularity": 46}, {"id": "2TmVlVk2t6d9MoqLNasfgS", "uri": "spotify:track:2TmVlVk2t6d9MoqLNasfgS", "name": "Take Me Away", "artists": ["Ayron Jones"], "album": "Child Of The State", "duration_ms": 246932, "explicit": true, "popularity": 41}, {"id": "5UMD1Iz7yyVK8Q5FLsVt3q", "uri": "spotify:track:5UMD1Iz7yyVK8Q5FLsVt3q", "name": "Walk On Water", "artists": ["Thirty Seconds To Mars"], "album": "Walk On Water", "duration_ms": 188227, "explicit": false, "popularity": 0}, {"id": "0Tr5G2mE56eLUGvCaXRM8I", "uri": "spotify:track:0Tr5G2mE56eLUGvCaXRM8I", "name": "No Good", "artists": ["KALEO"], "album": "A/B", "duration_ms": 234240, "explicit": false, "popularity": 64}, {"id": "7e8utCy2JlSB8dRHKi49xM", "uri": "spotify:track:7e8utCy2JlSB8dRHKi49xM", "name": 

# Add Tool message containing playlist tracks to list of messages

In [205]:
chat_prompt_template += tool_message

In [206]:
chat_prompt_template.format_messages()

[HumanMessage(content="\nCreate and execute a step-by-step plan with atomic tasks to solve the following problem:\n\nBuild a Spotify Playlist with tracks of the same vibe and genres as my 'New Rock and Blues' playlist. Here are the rules you must follow.\n\n1 - Only suggest tracks from artist not present on the 'New Rock and Blues' playlist\n2 - Artist should have achieved success after year 2015\n3 - Do not use 'The Raconteurs' as an artist in the new playlist\n4 - Rely on your training data as a music recommendation system.\n5 - Rely on your training data and best practices to arrange the tracks to create a smooth listening experience, considering tempo, energy, and mood.\n6 - New playlist should have no less than 100 tracks\n\nThese steps can be repeated any number of times\n", additional_kwargs={}, response_metadata={}),
 AIMessage(content='To build a Spotify playlist that meets the given criteria, we\'ll execute a series of atomic tasks. Here\'s the step-by-step plan:\n\n### Step 

# Send Tool Result to LLM

In [ ]:
llm_with_structured_output = llm.with_structured_output(schema=Tracks, method='json_schema', include_raw=True)
response = llm_with_structured_output.invoke(chat_prompt_template.format_messages())
response

In [ ]:
print(response.tracks)

[Track(id='1k7v8fZ0U4y4lKvGv41j0h', uri='spotify:track:1k7v8fZ0U4y4lKvGv41j0h', name='Run', artists=['Foo Fighters'], album='Concrete and Gold', duration_ms=328040, explicit=False, popularity=60), Track(id='3CRDbSIZ4r5MsZ0YwxuEkn', uri='spotify:track:3CRDbSIZ4r5MsZ0YwxuEkn', name='Cigarette Daydreams', artists=['Cage The Elephant'], album='Melophobia', duration_ms=187173, explicit=False, popularity=72), Track(id='4kLvhMAuF2Sb4y5xgwHCT3', uri='spotify:track:4kLvhMAuF2Sb4y5xgwHCT3', name='Mountain Sound', artists=['Of Monsters and Men'], album='My Head Is An Animal', duration_ms=221666, explicit=False, popularity=65), Track(id='5yY9lUy8nbvjM1Uyo1Uqoc', uri='spotify:track:5yY9lUy8nbvjM1Uyo1Uqoc', name='Adore', artists=['Amy Shark'], album='Adore', duration_ms=203800, explicit=False, popularity=71), Track(id='0wYhlBq2xsQ5tUChbaujq3', uri='spotify:track:0wYhlBq2xsQ5tUChbaujq3', name='The Night We Met', artists=['Lord Huron'], album='Strange Trails', duration_ms=214866, explicit=False, popul